## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## План выполнению: ##
### Шаг 1. Подгрузить данные ###
    Путь к файлу: /datasets/data.csv. 
### Шаг 2. Предобработка данных ###
1. определите и заполните пропущенные значения:
    - опишите, какие пропущенные значения вы обнаружили;
    - приведите возможные причины появления пропусков в данных;
    - объясните, по какому принципу заполнены пропуски;
3. замените вещественный тип данных на целочисленный:
    - поясните, как выбирали метод для изменения типа данных;
4. удалите дубликаты:
    - поясните, как выбирали метод для поиска и удаления дубликатов в данных;
    - приведите возможные причины появления дубликатов;
5. выделите леммы в значениях столбца с целями получения кредита:
    - опишите, как вы проводили лемматизацию целей кредита;
6. категоризируйте данные:
    - перечислите, какие «словари» вы выделили для этого набора данных, и объясните, почему.
    - В данных могут встречаться артефакты — значения, которые не отражают действительность. Например, отрицательное количество дней трудового стажа. Для реальных данных — это нормально. Нужно описать возможные причины появления таких данных и обработать их.
    
### Шаг 3. Ответить на вопросы ###
1. Есть ли зависимость между наличием детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

### Шаг 4. Общий вывод ###

### Описание данных: ###

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Анализ: ##
### Шаг 1. Подгрузить данные  ###

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info() 
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print('Количество пропусков категории "общий трудовой стаж в днях": ', data['days_employed'].isna().sum())
print('Количество пропусков категории "ежемесячный доход": ', data['total_income'].isna().sum())



total_income_median = data['total_income'].median()
print('Медиана по ежемесячному доходу: ', total_income_median)

data['total_income'] = data['total_income'].fillna(value=total_income_median)
print(data['total_income'].head(13))

data['days_employed'] = data['days_employed'].fillna(value=0)
print(data['days_employed'].head(13))

data.info()

Количество пропусков категории "общий трудовой стаж в днях":  2174
Количество пропусков категории "ежемесячный доход":  2174
Медиана по ежемесячному доходу:  145017.93753253992
0     253875.639453
1     112080.014102
2     145885.952297
3     267628.550329
4     158616.077870
5     255763.565419
6     240525.971920
7     135823.934197
8      95856.832424
9     144425.938277
10    113943.491460
11     77069.234271
12    145017.937533
Name: total_income, dtype: float64
0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12         0.000000
Name: days_employed, dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int6

### Вывод

1) Нашла количество пропущенных значений по 2 столбцам - ежмесячный доход и общий трудовой стаж в днях.
2) По таблице данных выявила, что эти 2 параметра - сопутствующие, т.е. если нет первого, то нет и второго. (нет = пропуск)
3) Для замены пропущенных значений нашела медиану по среднему доходу.
4) Проверила, заменились ли пропуски медианой (для этого вывела первые 13 строк, т.е. до первой строки, где был пропуск в ячейке со средним доходом)
5) Заменила пропущенные значения в стаже на 0.
6) Проверила, заменились ли пропуски в столбце со стажем на 0.

Выводы: Пропуски в этих столбцах могут говорить о том, что данные люди еще не устроились на работу, следовательно, их стаж равен 0, но, вероятно, эти люди получают деньги не за работу, а какими-то иными способами (возможно, это временная подработка/помощь членов семьи и т.д.). Следовательно, вместо пропусков необходимо вставить медиану (не среднее арифметическое, т.к. при достаточно большом разбросе среднего дохода медиана лучше отражает состояние по данному критерию). Стаж изменяем на 0, т.к. данный показатель не является целевым в нашем исследовании (цель исследования - выявить, как семейное положение и количество детей влияет на факт погашения кредита в срок), но надо избавиться от пропусков. (К тому же сами данные по стажу некорректные, поэтому этот показатель нельзя исследовать пока данные не будут приведены в корректный вид - без отрицательных значений, без слишком больших значений (стаж в 130 лет) и т.д.).

P.S. Не совсем поняла, почему значения в столбце трудовой стаж в таблице отрицательные. Мне кажется, что данные некорректные по этому столбцу: помимо отрицательных чисел (отрицательные числа, возможно, появились из-за того, что в эту базу данных сливались данные с разных источников, поэтому формулы по рассчету стажа различались) также встречаются слишком большие числа (например, 400281.1369125588 дней у 19-ого по списку) - вряд ли возможно иметь такой стаж в пределах одной жизни. Возможно, стаж здесь измеряется не в днях, а в минутах, тогда эта цифра кажется правдоподобной (стаж примерно 2,3 года). Также, возможно, гипотеза, что люди, у которых пропущена ячейка со среднемесячным доходом, не работают не самом деле является ложной, т.к. в графе тип занятости у этих людей стоят, например, "сотрудник" или "госслужащий". В таком случае наши пропуски - это случайно пропущенные значения. Поэтому, возможно, стоит сделать следующие действия: 1) рассчитать медиану в каждой группе из 3 - госслужащие, сотрудники, компаньоны. 2) заменить на соответствующие значения (пропуски у сотрудников заменить на медиану по всем сотрудникам, пропуски у госслужащих - медианой у госслужащих, пропуски у компаньонов - медианой у компаньонов). 3) Заменить доход пенсионеров на 0, т.к. они уже не работают. (Хотелось бы получить комментарий по этим 3 пунктам. 

### Замена типа данных

In [3]:

data.info()

data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].astype('int', errors='ignore')
data.info()

print(data['days_employed'].head())
print(data['total_income'].head())

print(data['days_employed'].sum())
print(data['total_income'].sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status

### Вывод

1) Через info() выявила, что у общего трудового стажа и ежемесячного дохода тип данных float.
2) Для стажа в днях нам достаточно иметь количество целых дней, поэтому заменяем на целочисленный тип данных.
3) Для среднемесячного дохода тоже нет необходимости знать в точности до копеек, т.к. в дальнейшем, скорее всего, будет использоваться кластеризация, например, с шагом 5000. Следовательно, тоже заменяем на целочисленный тип данных.
4) Проверяем, изменился ли тип данных, выводим первые 5 строк.
5) Проверяем возможность совершать математические действия. Ошибки не выявлены, следовательно, изменение типов данных прошло успешно.

### Обработка дубликатов

In [4]:

print(data.duplicated().sum())

print(data['education'].unique())
print(data['education'].value_counts())

data['education'] = data['education'].str.lower()
print(data['education'].unique())
print(data['education'].value_counts())

print(data['family_status'].unique())
data['family_status'] = data['family_status'].str.lower()
print(data['family_status'].unique())

print(data['income_type'].unique())

print(data.duplicated().sum())

data = data.drop_duplicates().reset_index(drop=True)
print(data.duplicated().sum())

54
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Н

### Вывод

1) Ищем количество дубликатов - 54.
2) Далее путем ручного анализа информации выявила, что в столбце образование попадаются значения заглавными буквами. Для проверки вызываю unique() и value_counts(). Действительно в таблице встречаются, например, такие значения: "Среднее", "среднее", "СРЕДНЕЕ", которые считаются как разные значения, хотя на самом деле означают одно значение.
3) Приводим к нижнему регистру и проверяем уникальные значения. Их стало меньше. 
4) Далее проверяю другие показатели. по семейному статусу нет повторений, но для удобства "Не женат / не замужем" делаю в нижнем регистре, т.к. все остальные значения в данном столбце тоже в нижнем регистре.
5) После приведения этих показателей в нижний регистр проверяю количество дубликатов. Теперь их стало немного больше - 71.
6) Удаляю дубликаты с обновлением индексов и удалением старых. Проверяю количество дубликатов в итоговой таблице - 0, следовательно, дубликаты успешно удалены.

Выводы:
Возможно, дубликаты появляются из-за того, что в базу данных "сливались" данные с разных источников, и те или иные люди могли быть зафиксированы в разных базах данных. 

### Лемматизация

In [5]:

from pymystem3 import Mystem
m = Mystem() 

def purpose_lemmas(purpose):
    return m.lemmatize(purpose)

data['purpose_lemmas'] = data['purpose'].apply(purpose_lemmas)

display(data)

print(data['purpose_lemmas'].value_counts())


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]"
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]"
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]"
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]"


[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

### Вывод

1) Создаем функцию, которая будет делать лемматизацию целей кредита.
2) Сохраняем результаты лемматизации в новом столбце - purpose_lemmas.
3) Выводим результат, новый столбец создан, данные корректны.
4) Подсчитываем количество каждой леммы, чтобы выявить наиболее часто встречающиеся слова.

### Категоризация данных

In [6]:
def add_purpose_categ(purpose):
    if 'автомобиль' in m.lemmatize(purpose):
        return 'автомобиль'
    elif 'свадьба' in m.lemmatize(purpose):
        return 'свадьба'
    elif 'жилье' in m.lemmatize(purpose) or 'недвижимость' in m.lemmatize(purpose):
        return 'жилье'
    return 'образование'
data['purpose_category'] = data['purpose'].apply(add_purpose_categ)
print(data['purpose_category'].value_counts())

def add_income_category(total_income):
    if total_income < 60000:
        return 'низкий уровень достатка'
    elif 60000 <= total_income <= 150000:
        return 'средний уровень достатка'
    return 'высокий уровень достатка'
data['income_category'] = data['total_income'].apply(add_income_category)
print(data['income_category'].value_counts())

print(data['family_status'].value_counts())
def add_family_status_category (family_status):
    if 'женат / замужем' in family_status or 'гражданский брак' in family_status:
        return 'есть пара'
    return 'одинок(-а)'
data['family_status_category'] = data['family_status'].apply(add_family_status_category)
print(data['family_status_category'].value_counts())

print(data['children'].value_counts())
def add_children_category (children):
    if children == 0:
        return 'бездетные'
    elif 1<= children <=2:
        return 'малодетные'
    elif 3 <= children <=5:
        return 'многодетные'
    return 'некорректные данные или статистические выбросы'
data['children_category'] = data['children'].apply(add_children_category)   
print(data['children_category'].value_counts())


display(data.head(20))

жилье          10811
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_category, dtype: int64
средний уровень достатка    11464
высокий уровень достатка     9184
низкий уровень достатка       806
Name: income_category, dtype: int64
женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64
есть пара     16490
одинок(-а)     4964
Name: family_status_category, dtype: int64
 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
бездетные                                         14091
малодетные                                         6860
многодетные                                         380
некорректные данные или статистические выбросы      123
Name: children_category, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmas,purpose_category,income_category,family_status_category,children_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",жилье,высокий уровень достатка,есть пара,малодетные
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,средний уровень достатка,есть пара,малодетные
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",жилье,средний уровень достатка,есть пара,бездетные
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий уровень достатка,есть пара,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,высокий уровень достатка,есть пара,бездетные
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",жилье,высокий уровень достатка,есть пара,бездетные
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",жилье,высокий уровень достатка,есть пара,бездетные
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,"[образование, \n]",образование,средний уровень достатка,есть пара,бездетные
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,средний уровень достатка,есть пара,малодетные
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",жилье,средний уровень достатка,есть пара,бездетные


### Вывод

1) Создаю категории по критерию цели кредита (исходя из лемматизации, выделяю 4 наиболее распространенные категории - образование, жилье, автомобиль, свадьба).
2) Также создаю категории по критерию семейного положения, т.к. цель исследования - выявить влияние семейного положения и количества детей на факт погашения кредита. (По семейному положению объединил тех, кто в официальном и гражданском браке в категорию "есть пара", а остальных (не женат/не замужем, в разводе, вдовец/вдова) - в "одинок(-а))
3) По количеству детей: бездетных больше всего в выборке, малодетных клиентов (1-2) ребенка объединил в 1 категорию, больше 3 - многодетные (я поставила границу в 5, т.к. в выборке не встречается клиентов, у которых больше 5)*. Но есть отрицательные значения и значения "20 детей" - первые точно являются некорректными данными, а вторые - скорее всего, тоже, но даже если предположить, что у кого-то действительно может быть 20 детей, то такие клиенты будут статистическими выбросами и могут не учитываться из-за малого количества.*
4) По критерию дохода также создала 3 категории - высокий, средний и низкий уровень дохода.

### Шаг 3. Ответить на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
# Убрала все лишнее (Код с прошлых шагов). Теперь здесь только код по ответу на вопрос о наличии зависимости между наличием детей и возвратом кредита в срок.

data['children_category'] = data['children'].apply(add_children_category)
data_pivot_children = data.pivot_table(index='children_category', columns='debt', values='children', aggfunc='count')
data_pivot_children['sum'] = data_pivot_children[0] + data_pivot_children[1]
data_pivot_children['result %'] = (data_pivot_children[1] / data_pivot_children['sum']) * 100
display(data_pivot_children.sort_values(by='result %'))

debt,0,1,sum,result %
children_category,,,,
некорректные данные или статистические выбросы,114,9,123,7.317073
бездетные,13028,1063,14091,7.543822
многодетные,349,31,380,8.157895
малодетные,6222,638,6860,9.300292


### Вывод

1) Создаю сводную таблицу по категориям бездетных, малодетных и многодетных детей (категории выделял на прошлом шаге).
2) Считаю общее количество показателей в столбце sum и процент тех, у кого была задолженность по кредитам.

Выводы: Среди бездетных наименьший процент тех, кто не выплачивал кредит в срок. Однако по данной выборке нельзя утверждать, что с ростом количества детей, растет вероятность невыплаты кредитов, т.к. среди многодетных данный показатель ниже, чем у малодетных. Вероятно, что выборка не совсем репрезентативная, т.к. клиентов с многодетными семьями всего 380. Возможно, при большем количестве многодетных семей мы бы получили, закономерность, что с ростом количества детей растет риск невыплаты кредита в срок. Предположу, что такая тенденция проявляется из-за того, что у бездетных клиентов больше возможностей посвящать все свое время работе и меньше непредвиденных расходов на дополнительных членов семьи, чем у семей с детьми, поэтому в категории бездетных реже всего не выплачивается кредит в срок. (Категорию "некорректные данные или статистические выбросы не учитываем, т.к. в ней значения показателя "количество детей" - "-1" или "20".)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
# Убрал все лишнее (Код с прошлых шагов). Теперь здесь только код по вопросу о зависимость между семейным положением и возвратом кредита в срок.
data['family_status_category'] = data['family_status'].apply(add_family_status_category)
data_pivot_family_status = data.pivot_table(index='family_status_category', columns='debt', values='family_status', aggfunc='count')
data_pivot_family_status['sum'] = data_pivot_family_status[0] + data_pivot_family_status[1]
data_pivot_family_status['result %'] = (data_pivot_family_status[1] / data_pivot_family_status['sum']) * 100
display(data_pivot_family_status.sort_values(by='result %'))



debt,0,1,sum,result %
family_status_category,,,,
есть пара,15171,1319,16490,7.998787
одинок(-а),4542,422,4964,8.501209


### Вывод

1) Построила сводную таблицу по категориям семейного положения (выделил на прошлом шаге 2 категории: "женат/замужем" и "гражданском браке" объединил в категорию "есть пара"; остальные - "в разводе", "вдовец/вдова", "не женат/не замужем" - в категорию "одинок(-а)").
2) Рассчитала сумму и процент тех, кто имел задолженность по кредиту. 

Вывод: по выделенным категориям и по данной выборке можно утверждать, что у людей, у которых есть пара, вероятность выплаты кредита в срок выше, чем у людей, которые одиноки. Скорее всего, такая ситуация складывается из-за "двойного дохода" - работают оба члена семьи. (расходы в этом случае на коммунальные услуги, расходы на питание, аренду и т.д. растут не пропорционально, следовательно, те, у кого есть пара, могут больше денег откладывать на выплату кредита в срок).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
data['income_category'] = data['total_income'].apply(add_income_category)
data_pivot_income_category = data.pivot_table(index='income_category', columns='debt', values='total_income', aggfunc='count')
data_pivot_income_category['sum'] = data_pivot_income_category[0] + data_pivot_income_category[1]
data_pivot_income_category['result %'] = (data_pivot_income_category[1] / data_pivot_income_category['sum']) * 100
display(data_pivot_income_category.sort_values(by='result %'))



debt,0,1,sum,result %
income_category,,,,
низкий уровень достатка,757,49,806,6.079404
высокий уровень достатка,8458,726,9184,7.905052
средний уровень достатка,10498,966,11464,8.426378


### Вывод

1) По выделенным категориям дохода создаем сводную таблицу. 
2) Рассчитываем количество измерений по категориям и процент тех, кто не выплачивал кредит в срок.

Вывод: по данной выборке нельзя утверждать, что зависимость между уровнем дохода и возвратом кредита в срок есть. Люди с низким уровнем дохода чаще всего выплачивают кредит в срок (но выборка слишком непропорциональна - людей с низким уровнем дохода в данной базе мало, чтобы делать точные выводы). Возможно, это из-за того, что они не берут больших кредитов и им при необходимости относительно легче найти деньги (например, занять у родственников, т.к. сумма не очень большая), чтобы погасить кредит в срок. Но люди с высоким уровнем достатка имеют вероятность невыплаты кредита в срок меньше, чем люди со средним уровнем дохода. Прямой закономерности не обнаружено.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:

data['purpose_category'] = data['purpose'].apply(add_purpose_categ)
data_pivot_purpose_category = data.pivot_table(index='purpose_category', columns='debt', values='purpose', aggfunc='count')
data_pivot_purpose_category['sum'] = data_pivot_purpose_category[0] + data_pivot_purpose_category[1]
data_pivot_purpose_category['result %'] = (data_pivot_purpose_category[1] / data_pivot_purpose_category['sum']) * 100
display(data_pivot_purpose_category.sort_values(by='result %'))





### Вывод

1) Строим сводную таблицу по 4 выделенным категориям - жилье, образование, свадьба, автомобиль.
2) Рассчитываем общее количество измерений по каждой категории и процент тех, кто не выплачивал кредит в срок.

Вывод: из приведенных 4 категорий меньше всего задерживают выплаты те, кто берет кредит на жилье. Клиенты, которые берут кредит на образование и покупку автомобиля, чаще всего не возвращают кредит в срок. Примерно промежуточное положение занимают клиенты, которые берут кредит на проведение свадьбы.

### Шаг 4. Общий вывод

Таким образом, мы можем ответить на 2 главных вопроса:
1) Влияет ли семейное положение на факт погашения кредита в срок?
Да, клиенты, которые имеют пару, чаще выплачивают кредиты в срок, чем те, кто живет один. (Возможные причины указал ранее (про "двойной доход"))
2) Влияет ли количество детей на факт погашения кредита в срок?
Отчасти влияет: бездетные клиенты чаще выплачивают кредит в срок, однако нельзя утверждать, что чем больше детей, тем ниже вероятность выплаты кредита в срок, т.к. многодетные выплачивают кредит в срок чаще, чем малодетные.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Выводы понятны и логичны, а самое главное подкреплены выявленными фактами.
       
В целом сам проект выполнен на хорошем уровне! На протяжении всей работы чувствуется глубина проработки задачи и это безусловный плюс для нас. Было использовано большое количество методов, которые будут помогать тебе в дальнейших проектах.
    
Если вдруг у тебя есть какие–то вопросы, то я с радостью отвечу на них. 
    
</div>